In [40]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [41]:
! dvc pull

  8.00k/1.35M [00:00<01:10,    19.9kB/s]


  0%|          |model.pkl                 8.00k/2.54M [00:00<02:08,    20.6kB/s]

  0%|          |data/prepared/train.tsv   8.00k/18.2M [00:00<18:14,    17.4kB/s]
 17%|█▋        |data/features/test.pkl     240k/1.35M [00:00<00:41,    28.3kB/s]




  0%|          |data/data.xml             8.00k/36.1M [00:00<31:58,    19.7kB/s]



  0%|          |data/features/train.pkl   8.00k/5.41M [00:00<05:39,    16.7kB/s]





  0%|          |data/prepared/test.tsv    8.00k/4.54M [00:00<04:06,    19.3kB/s]


  4%|▍         |model.pkl                  104k/2.54M [00:00<01:27,    29.1kB/s]

  0%|          |data/prepared/train.tsv   80.0k/18.2M [00:00<12:51,    24.7kB/s]
 45%|████▌     |data/features/test.pkl     624k/1.35M [00:00<00:19,    40.2kB/s]




  0%|          |data/data.xml             80.0k/36.1M [00:00<22:41,    27.8kB/s]

  1%|          |data/prepared/train.tsv    120k/18.2M [00:00<09:13,    34.3kB/s]





  1%|          |data/prepared/test.tsv  

## Easily reproduce somewhere else

### from cache

In [42]:
! pipenv run dvc repro

Loading .env environment variables...
If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Stage 'data/data.xml.dvc' didn't change, skipping
Stage 'prepare' didn't change, skipping
Stage 'featurize' didn't change, skipping
Stage 'train' didn't change, skipping
Stage 'evaluate' didn't change, skipping
Data and pipelines are up to date.


### or force actual pipeline run

In [43]:
! dvc repro -f

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Verifying data sources in stage: 'data/data.xml.dvc'

Running stage 'prepare' with command:
	python src/prepare.py data/data.xml

Running stage 'featurize' with command:
	python src/featurization.py data/prepared data/features
The input data frame data/prepared/train.tsv size is (20017, 3)
The output matrix data/features/train.pkl size is (20017, 502) and data type is float64
The input data frame data/prepared/test.tsv size is (4983, 3)
The output matrix data/features/test.pkl size is (4983, 502) and data type is float64

Running stage 'train' with command:
	python src/train.py data/features model.pkl
Input matrix size (20017, 502)
X matrix size (20017, 500)
Y matrix size (20017,)
If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>
Running stage 'evaluate' with command:
	python src/evaluate.py model.pkl data/features scores.json prc.json
If DVC froze, see `hardlink_lock` in <https://man.dvc.org/con

In [44]:
! dvc diff

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>

### Assert equivalent result

In [45]:
! cat scores.json

{"auc": 0.5417487597055675}

### Now I can easily
* Making updates to the models
* see differences in parameters or results
* collaborate with others
* reproduce results

### Go and modify some parameters

In [46]:
! dvc status

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Data and pipelines are up to date.


In [47]:
! dvc params diff

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>

In [48]:
! dvc repro

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Stage 'data/data.xml.dvc' didn't change, skipping
Stage 'prepare' didn't change, skipping
Stage 'featurize' didn't change, skipping
Stage 'train' didn't change, skipping
Stage 'evaluate' didn't change, skipping
Data and pipelines are up to date.


In [49]:
! dvc metrics diff

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>

In [50]:
! cat scores.json

{"auc": 0.5417487597055675}

In [51]:
! dvc plots diff -x recall -y precision

file:///home/nichita/Projects/dvc_intro/plots.html


## For reproducibility of this notebook:
### Remove tracked data from dvc (local and remote) and
### Remove unnecessary files 

In [52]:
# ! dvc remove data/data.xml.dvc
! dvc remove data/.
! rm -rf data/
! rm plots.html
! rm prc.json
! rm model.pkl

! rm data/data.xml

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>

In [53]:
! dvc diff

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Deleted:
    data/data.xml

files summary: 0 added, 1 deleted, 0 modified


In [54]:
! git add data/data.xml.dvc
! git commit -m 'removes data'
! git push origin tutorial
! dvc push

[data_pipelines 88021b5] removes data
 1 file changed, 3 deletions(-)
 delete mode 100644 data/data.xml.dvc
Everything up-to-date
Everything is up to date.


In [55]:
! echo "y" | dvc gc -cw

  0% Querying cache in 'gs://dvc_intro'|  |1.00/4.10k [00:00<11:10,  6.11file/s]

In [56]:
! echo 'y' | dvc destroy

This will destroy all information about your pipelines, all data files, as well as cache in .dvc/cache.
If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>

In [57]:
! git add .
! git commit -m "clean"
! git push origin tutorial

[data_pipelines 1aa05a3] clean
 11 files changed, 306 insertions(+), 541 deletions(-)
 delete mode 100644 .dvc/.gitignore
 delete mode 100644 .dvc/config
 delete mode 100644 .dvc/plots/confusion.json
 delete mode 100644 .dvc/plots/default.json
 delete mode 100644 .dvc/plots/scatter.json
 delete mode 100644 .dvc/plots/smooth.json
 delete mode 100644 data/.gitignore
 delete mode 100644 dvc.lock
 delete mode 100644 dvc.yaml
 delete mode 100644 prc.json
 rewrite run.ipynb (76%)
Everything up-to-date
